In [1]:
import pandas as pd
import json
import numpy as np
import math
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


pd.set_option('display.max_columns', None)

In [2]:
# loading in the JSON data
with open('../data/match_results.json','r') as f:
    match_res = json.load(f)
match_res[0]

{'city': 'Brisbane',
 'dates': '2017-01-13',
 'gender': 'male',
 'match_type': 'ODI',
 'outcome.runs': 92,
 'outcome.winner': 'Australia',
 'overs': 50,
 'player_of_match': 'MS Wade',
 'teams': 'Australia',
 'toss.decision': 'bat',
 'toss.winner': 'Australia',
 'umpires': 'MD Martell',
 'venue': 'Brisbane Cricket Ground, Woolloongabba',
 'matchid': '1000887'}

In [3]:
match_df = pd.DataFrame(match_res)
print("Complete Match Results JSON Shape ", match_df.shape)
match_df = match_df[match_df['gender']=='male']
print("Complete Match Results JSON Shape for Men's matches ", match_df.shape)
match_df = match_df.iloc[:,:21] # dropping columns containing supersub info since not relevant
match_df.head()

Complete Match Results JSON Shape  (4306, 42)
Complete Match Results JSON Shape for Men's matches  (3810, 42)


,city,dates,gender,match_type,outcome.runs,outcome.winner,overs,player_of_match,teams,toss.decision,toss.winner,umpires,venue,matchid,outcome.wickets,result,outcome.method,neutral_venue,match_type_number,outcome.result,outcome.eliminator
0,Brisbane,2017-01-13,male,ODI,92.0,Australia,50,MS Wade,Australia,bat,Australia,MD Martell,"Brisbane Cricket Ground, Woolloongabba",1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brisbane,2017-01-13,male,ODI,92.0,Australia,50,MS Wade,Pakistan,bat,Australia,C Shamshuddin,"Brisbane Cricket Ground, Woolloongabba",1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2017-01-15,male,ODI,NaN,Pakistan,50,Mohammad Hafeez,Australia,bat,Australia,CB Gaffaney,Melbourne Cricket Ground,1000889,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2017-01-15,male,ODI,NaN,Pakistan,50,Mohammad Hafeez,Pakistan,bat,Australia,P Wilson,Melbourne Cricket Ground,1000889,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Perth,2017-01-19,male,ODI,NaN,Australia,50,SPD Smith,Australia,field,Australia,SD Fry,Western Australia Cricket Association Ground,1000891,7.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
match_df.isna().sum()

city                   622
dates                    0
gender                   0
match_type               0
outcome.runs          2024
outcome.winner         190
overs                    0
player_of_match        248
teams                    0
toss.decision            0
toss.winner              0
umpires                  0
venue                    0
matchid                  0
outcome.wickets       1976
result                3630
outcome.method        3466
neutral_venue         3180
match_type_number     2750
outcome.result        3800
outcome.eliminator    3806
dtype: int64

In [5]:
match_df['result'].value_counts()

result
no result    152
tie           28
Name: count, dtype: int64

In [6]:
# Filter matches: Men’s ODI with a definitive winner
filtered_matches = match_df[
    (match_df['result'] != 'no result')
]

valid_match_ids = filtered_matches['matchid'].unique().tolist()

In [7]:
# Load innings data
with open('../data/innings_results.json', 'r') as f:
    innings_res = json.load(f)

innings_df = pd.DataFrame(innings_res)
# Filter by valid matches
innings_df = innings_df[innings_df['matchid'].isin(valid_match_ids)]
# keeping select columns
select_cols = ['runs.batsman','runs.extras','runs.total','over','team','innings','matchid','wides','wicket.kind','wicket.player_out','wicket.fielders','legbyes','noballs','byes']
innings_df = innings_df.loc[:,select_cols]
print("Dimensions after only keeping relevant matchids and cleaning columns", innings_df.shape)
innings_df.head()

Dimensions after only keeping relevant matchids and cleaning columns (1000742, 14)


,runs.batsman,runs.extras,runs.total,over,team,innings,matchid,wides,wicket.kind,wicket.player_out,wicket.fielders,legbyes,noballs,byes
0,0,0,0,0.1,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0.2,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.3,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0.4,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1,1,0.5,Australia,1,1000887,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Convert over to numeric for sorting and calculations
innings_df['over_numeric'] = innings_df['over'].astype(float)
innings_df = innings_df.sort_values(by=['matchid', 'innings', 'over_numeric']).reset_index(drop=True)
innings_df.head()

,runs.batsman,runs.extras,runs.total,over,team,innings,matchid,wides,wicket.kind,wicket.player_out,wicket.fielders,legbyes,noballs,byes,over_numeric
0,0,0,0,0.1,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1
1,0,0,0,0.2,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2
2,0,0,0,0.3,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3
3,0,0,0,0.4,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4
4,0,1,1,0.5,Australia,1,1000887,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.5


In [9]:
innings_df.isna().sum()

runs.batsman              0
runs.extras               0
runs.total                0
over                      0
team                      0
innings                   0
matchid                   0
wides                978067
wicket.kind          972969
wicket.player_out    972969
wicket.fielders      982227
legbyes              990417
noballs              996855
byes                 999290
over_numeric              0
dtype: int64

In [10]:
innings_df['wicket.kind'].value_counts()

wicket.kind
caught                   15463
bowled                    4891
lbw                       2952
run out                   2882
caught and bowled          806
stumped                    711
retired hurt                38
hit wicket                  24
obstructing the field        6
Name: count, dtype: int64

In [11]:
# checking retired hurt values
hurt_index = innings_df[innings_df['wicket.kind']=='retired hurt'].index[0]
innings_df[hurt_index-2:hurt_index+2]

,runs.batsman,runs.extras,runs.total,over,team,innings,matchid,wides,wicket.kind,wicket.player_out,wicket.fielders,legbyes,noballs,byes,over_numeric
2811,2,0,2,30.4,Pakistan,2,1000895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.4
2812,1,0,1,30.5,Pakistan,2,1000895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.5
2813,0,0,0,30.6,Pakistan,2,1000895,NaN,retired hurt,Shoaib Malik,NaN,NaN,NaN,NaN,30.6
2814,0,0,0,30.7,Pakistan,2,1000895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.7


In [18]:
def process_innings_data(data):
    wickets_fallen = 0
    innings_processed = []
    wicket_tags = ['caught', 'bowled', 'lbw', 'run out', 'caught and bowled', 'stumped', 'hit wicket', 'obstructing the field']
    total_overs = 50
    total_balls = total_overs * 6
    legal_deliveries_bowled = 0
    
    for i, row in data.iterrows():
        runs_on_ball = row.get('runs.total', 0)

        wicket_on_ball = 1 if (row['wicket.kind'] in wicket_tags) else 0
        wickets_remaining = 10 - wickets_fallen

        # Determine if this ball is legal
        # If it's a wide or no-ball, do not increment legal_deliveries_bowled
        legal = True if (math.isnan(row['wides']) & math.isnan(row['noballs'])) else False
        if legal:
            legal_deliveries_bowled += 1

        # Calculate remaining overs in standard cricket format (O.B)
        balls_remaining = total_balls - legal_deliveries_bowled
        overs_remaining_int = balls_remaining // 6
        balls_remaining_in_over = balls_remaining % 6
        remaining_overs = overs_remaining_int + (balls_remaining_in_over * 0.1)

        innings_processed.append({
            'matchid': row['matchid'],
            'team': row['team'],
            'innings': row['innings'],
            'remaining_overs': remaining_overs,
            'remaining_wickets': wickets_remaining,
            'runs_on_ball': runs_on_ball,
            'wicket_on_ball': wicket_on_ball,
        })

        if wicket_on_ball:
            wickets_fallen += 1

    return pd.DataFrame(innings_processed)

# Apply to each innings
final_balls_df = innings_df.groupby(['matchid', 'team', 'innings'], group_keys=False)[list(innings_df.columns)].apply(process_innings_data).reset_index(drop=True)

In [19]:
final_balls_df.head(20)

,matchid,team,innings,remaining_overs,remaining_wickets,runs_on_ball,wicket_on_ball
0,1000887,Australia,1,49.5,10,0,0
1,1000887,Australia,1,49.4,10,0,0
2,1000887,Australia,1,49.3,10,0,0
3,1000887,Australia,1,49.2,10,0,0
4,1000887,Australia,1,49.2,10,1,0
5,1000887,Australia,1,49.1,10,0,0
6,1000887,Australia,1,49.0,10,0,0
7,1000887,Australia,1,48.5,10,0,0
8,1000887,Australia,1,48.4,10,1,0
9,1000887,Australia,1,48.3,10,0,0


In [14]:
# Get sum of runs per over
final_balls_df['remaining_overs'] = final_balls_df['remaining_overs'].apply(lambda r:int(r))
over_data = final_balls_df.groupby(['matchid', 'team', 'innings', 'remaining_overs'], sort=False)['runs_on_ball'].sum().reset_index(name='runs_this_over')
# since questions is asking for runs per over, not runs per over with wickets in hand -- below code is commented out
# over_runs_df = final_balls_df.groupby(['matchid', 'team', 'innings', 'remaining_overs','remaining_wickets'], sort=False)['runs_on_ball'].sum().reset_index(name='runs_this_over')
over_data

,matchid,team,innings,remaining_overs,runs_this_over
0,1000887,Australia,1,49,1
1,1000887,Australia,1,48,1
2,1000887,Australia,1,47,3
3,1000887,Australia,1,46,6
4,1000887,Australia,1,45,2
...,...,...,...,...,...
163555,997995,United Arab Emirates,1,8,1
163556,997995,United Arab Emirates,1,7,1
163557,997995,United Arab Emirates,1,6,2
163558,997995,United Arab Emirates,1,5,3


In [15]:
over_data['team'].value_counts()

team
India                       18285
Australia                   16698
Sri Lanka                   16646
England                     15158
Pakistan                    14812
South Africa                13273
West Indies                 13036
New Zealand                 12286
Bangladesh                  11285
Zimbabwe                    10232
Ireland                      5204
Afghanistan                  4386
Scotland                     2592
United Arab Emirates         2031
Kenya                        1205
Canada                       1095
Netherlands                  1088
Papua New Guinea              830
Hong Kong                     758
United States of America      585
Oman                          493
Bermuda                       459
Nepal                         331
Namibia                       324
Asia XI                       200
Africa XI                     194
ICC World XI                   74
Name: count, dtype: int64

In [16]:
# Example: split by matchid
match_ids = over_data['matchid'].unique()

import random
random.seed(123)
np.random.shuffle(match_ids)
train_ids = match_ids[:int(0.8*len(match_ids))]
test_ids = match_ids[int(0.8*len(match_ids)):]

train_df = over_data[over_data['matchid'].isin(train_ids)]
test_df = over_data[over_data['matchid'].isin(test_ids)]

# train_df, test_df = train_test_split(over_data, test_size=0.3, random_state=42)

features = ['team','innings','remaining_overs']

X_train = train_df[features]
y_train = train_df['runs_this_over']

X_test = test_df[features]
y_test = test_df['runs_this_over']

In [17]:
categ_features = ['team']
num_features = ['innings','remaining_overs']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categ_features),
    ],
    remainder='passthrough'  # leaves numeric features as-is
)

# Build a pipeline: first transform (preprocessor), then fit a model
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5

print(f"MAE: {mae}, RMSE: {rmse}")

MAE: 2.7279652620612884, RMSE: 3.4850785437534495


In [25]:
#joblib.dump(model_pipeline, 'models/model.pkl')

['rf_model.pkl']

In [ ]:
#predicting runs
